In [91]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [92]:
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [93]:
from langchain_community.document_loaders import PyMuPDFLoader

In [82]:
loader = PyMuPDFLoader("DeepPoV-Generative-AI.pdf",mode='page')

In [83]:
pages = loader.load()

In [84]:
pages

[Document(metadata={'producer': 'Adobe PDF library 16.07', 'creator': 'Adobe Illustrator 26.3 (Macintosh)', 'creationdate': '2022-12-14T17:48:28+05:30', 'source': 'DeepPoV-Generative-AI.pdf', 'file_path': 'DeepPoV-Generative-AI.pdf', 'total_pages': 47, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2023-01-18T13:17:26+05:30', 'trapped': '', 'modDate': "D:20230118131726+05'30'", 'creationDate': "D:20221214174828+05'30'", 'page': 0}, page_content='Generative AI\nDeep Point of View'),
 Document(metadata={'producer': 'Adobe PDF library 16.07', 'creator': 'Adobe Illustrator 26.3 (Macintosh)', 'creationdate': '2022-12-14T17:48:28+05:30', 'source': 'DeepPoV-Generative-AI.pdf', 'file_path': 'DeepPoV-Generative-AI.pdf', 'total_pages': 47, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2023-01-18T13:17:26+05:30', 'trapped': '', 'modDate': "D:20230118131726+05'30'", 'creationDate': "D:20221214174828+05'30'", '

In [85]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [86]:
page_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=300)

In [87]:
splited_pages = page_splitter.split_documents(pages)

In [88]:
splited_pages

[Document(metadata={'producer': 'Adobe PDF library 16.07', 'creator': 'Adobe Illustrator 26.3 (Macintosh)', 'creationdate': '2022-12-14T17:48:28+05:30', 'source': 'DeepPoV-Generative-AI.pdf', 'file_path': 'DeepPoV-Generative-AI.pdf', 'total_pages': 47, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2023-01-18T13:17:26+05:30', 'trapped': '', 'modDate': "D:20230118131726+05'30'", 'creationDate': "D:20221214174828+05'30'", 'page': 0}, page_content='Generative AI\nDeep Point of View'),
 Document(metadata={'producer': 'Adobe PDF library 16.07', 'creator': 'Adobe Illustrator 26.3 (Macintosh)', 'creationdate': '2022-12-14T17:48:28+05:30', 'source': 'DeepPoV-Generative-AI.pdf', 'file_path': 'DeepPoV-Generative-AI.pdf', 'total_pages': 47, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2023-01-18T13:17:26+05:30', 'trapped': '', 'modDate': "D:20230118131726+05'30'", 'creationDate': "D:20221214174828+05'30'", '

In [89]:
from langchain_huggingface import HuggingFaceEmbeddings

In [90]:
embeddings = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")

In [ ]:
texts = [doc.page_content for doc in splited_pages]
embeddings.embed_documents(texts)

In [ ]:
from langchain.vectorstores import FAISS

In [ ]:
vectorestoredb = FAISS.from_documents(splited_pages,embeddings)

In [ ]:
retriever = vectorestoredb.as_retriever()

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(model="mistral-saba-24b")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder

In [ ]:
domain_prompts = {
    "Academic/Research": "You are a research assistant. Answer in an academic tone with citations where applicable.",
    "Legal": "You are a legal expert. Reference legal clauses and explain in layman's terms.",
    "Healthcare": "You are a medical AI. Provide medically accurate responses and include disclaimers when needed."
}
domain = "legal"

In [ ]:
system_prompt = domain_prompts.get(
    domain,
    "You are a helpful assistant. Answer only from the context provided \n\n{context} . If the answer is not present in the document, respond clearly that the information is not available."
)

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
    ("human","{input}"),

    ]
)

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

In [ ]:
chain = create_stuff_documents_chain(llm,prompt)

In [ ]:
rag_chain = create_retrieval_chain(retriever,chain)

In [ ]:
from langchain.memory import ChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

In [ ]:
session_store = {}
def get_session_history(session_id:str)->InMemoryChatMessageHistory:
    if session_id not in session_store:
        session_store[session_id] = InMemoryChatMessageHistory()
    return session_store[session_id]

In [ ]:
with_message_history = RunnableWithMessageHistory(
    runnable=rag_chain,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
config = {
    "configurable":{
        "session_id":"user11"
    }
}

In [ ]:
response = with_message_history.invoke({
    "input": "what was my previous prompt"
},config=config
)

In [ ]:
response['answer']